In [1]:
import warnings; warnings.simplefilter('ignore')

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
import keras

Using TensorFlow backend.


In [5]:
from keras import backend as K

In [6]:
from keras.models import Sequential

In [7]:
from keras.layers import Dense

In [8]:
from sklearn import model_selection

In [9]:
import math

In [10]:
import tensorflow as tf

In [11]:
import horovod.keras as hvd

In [12]:
df = pd.DataFrame()

In [13]:
df['distances'] = np.array([np.random.randint(1,1000) for _ in range(10**5)])

In [14]:
df['weights'] = np.array([np.random.randint(60,500) for _ in range(10**5)])

In [15]:
df[['distances','weights']].count()

distances    100000
weights      100000
dtype: int64

In [16]:
df['prices'] = np.array([((df['distances'][i] * df['weights'][i])/100) for i in range(len(df['distances']))])

In [17]:
df.head()

,distances,weights,prices
0,869,258,2242.02
1,499,339,1691.61
2,684,287,1963.08
3,937,162,1517.94
4,982,378,3711.96


In [18]:
hvd.init()

0

In [19]:
config = tf.ConfigProto()

In [20]:
config.gpu_options.allow_growth = True

In [21]:
config.gpu_options.visible_device_list = str(hvd.local_rank())

In [22]:
K.set_session(tf.Session(config=config))

In [23]:
batch_size = 128

In [24]:
num_classes = 10

In [25]:
epochs = int(math.ceil(120.0) / hvd.size())

In [26]:
x, x_val, y, y_val = model_selection.train_test_split(df['prices'], df['distances'], test_size=0.2, random_state=42)

In [27]:
dlce = lambda l: [v for v in l if v >= 0]

In [28]:
x = np.array(dlce(x))
x_val = np.array(dlce(x_val))

In [29]:
y = np.array(dlce(y))
y_val = np.array(dlce(y_val))

In [30]:
opt = keras.optimizers.RMSprop(1.0 * hvd.size())

In [31]:
opt = hvd.DistributedOptimizer(opt)

In [32]:
model = Sequential()

In [33]:
model.add(Dense(1, input_dim=1))

In [34]:
model.compile(optimizer=opt, loss=keras.losses.mse,metrics=['accuracy'])

In [35]:
callbacks = [hvd.callbacks.BroadcastGlobalVariablesCallback(0)]

In [36]:
if hvd.rank() == 0:
    callbacks.append(keras.callbacks.ModelCheckpoint('./checkpoint-{epoch}.h5'))

In [37]:
model.fit(x, y, batch_size=batch_size, callbacks=callbacks, epochs=epochs, verbose=1, validation_data=(x_val, y_val))

Train on 80000 samples, validate on 20000 samples
Epoch 1/120
80000/80000 [==============================] - 1s 7us/step - loss: 824394.2524 - acc: 0.0014 - val_loss: 1603402.1048 - val_acc: 4.0000e-04
Epoch 2/120
80000/80000 [==============================] - 1s 7us/step - loss: 814417.5939 - acc: 9.1250e-04 - val_loss: 1753634.0944 - val_acc: 3.0000e-04
Epoch 3/120
80000/80000 [==============================] - 1s 7us/step - loss: 810519.6901 - acc: 5.7500e-04 - val_loss: 1427224.4748 - val_acc: 6.5000e-04
Epoch 4/120
80000/80000 [==============================] - 1s 7us/step - loss: 801345.9573 - acc: 8.2500e-04 - val_loss: 685139.4542 - val_acc: 4.0000e-04
Epoch 5/120
80000/80000 [==============================] - 1s 7us/step - loss: 809658.1291 - acc: 6.6250e-04 - val_loss: 422500.4696 - val_acc: 6.5000e-04
Epoch 6/120
80000/80000 [==============================] - 1s 7us/step - loss: 794994.8113 - acc: 6.0000e-04 - val_loss: 1452566.1584 - val_acc: 4.0000e-04
Epoch 7/120
80000/80

80000/80000 [==============================] - 1s 8us/step - loss: 808572.5777 - acc: 6.2500e-04 - val_loss: 524245.2173 - val_acc: 3.5000e-04
Epoch 54/120
80000/80000 [==============================] - 1s 6us/step - loss: 801458.4394 - acc: 6.2500e-04 - val_loss: 1351536.5432 - val_acc: 3.0000e-04
Epoch 55/120
80000/80000 [==============================] - 1s 7us/step - loss: 808087.9088 - acc: 5.1250e-04 - val_loss: 135429.2045 - val_acc: 0.0014
Epoch 56/120
80000/80000 [==============================] - 0s 6us/step - loss: 793869.2696 - acc: 7.5000e-04 - val_loss: 445909.7394 - val_acc: 6.0000e-04
Epoch 57/120
80000/80000 [==============================] - 1s 6us/step - loss: 789038.1195 - acc: 6.5000e-04 - val_loss: 607166.2219 - val_acc: 3.0000e-04
Epoch 58/120
80000/80000 [==============================] - 1s 6us/step - loss: 804091.8019 - acc: 6.2500e-04 - val_loss: 113801.5264 - val_acc: 9.5000e-04
Epoch 59/120
80000/80000 [==============================] - 1s 7us/step - loss: 

80000/80000 [==============================] - 1s 7us/step - loss: 806379.6972 - acc: 5.1250e-04 - val_loss: 266826.1553 - val_acc: 7.0000e-04
Epoch 106/120
80000/80000 [==============================] - 1s 7us/step - loss: 811273.3656 - acc: 5.1250e-04 - val_loss: 347898.1478 - val_acc: 3.5000e-04
Epoch 107/120
80000/80000 [==============================] - 1s 7us/step - loss: 801808.9733 - acc: 7.5000e-04 - val_loss: 740618.0483 - val_acc: 3.0000e-04
Epoch 108/120
80000/80000 [==============================] - 1s 7us/step - loss: 798192.7549 - acc: 7.7500e-04 - val_loss: 964829.1599 - val_acc: 6.0000e-04
Epoch 109/120
80000/80000 [==============================] - 1s 7us/step - loss: 804560.4711 - acc: 7.7500e-04 - val_loss: 573217.8133 - val_acc: 7.5000e-04
Epoch 110/120
80000/80000 [==============================] - 1s 7us/step - loss: 803573.1071 - acc: 6.5000e-04 - val_loss: 916490.9462 - val_acc: 2.5000e-04
Epoch 111/120
80000/80000 [==============================] - 1s 7us/step

In [38]:
score = model.evaluate(x_val, y_val, verbose=0)

In [39]:
print('Test loss:', score[0])

Test loss: 1943180.767


In [40]:
print('Test accuracy:', score[1])

Test accuracy: 0.00065


In [41]:
print(model.predict(np.array([np.random.randint(10,100) for _ in range(5)])))

[[ 205.49256897]
 [ 217.12568665]
 [ 207.23753357]
 [ 172.33816528]
 [ 172.33816528]]


In [42]:
model.save_weights('travel-price.hdf5')

In [43]:
with open('travel-price.json','w') as f:
    f.write(model.to_json())
    f.close()